# 0. Task description

### Bike Sharing Demand

You are provided **hourly rental data** spanning two years. For this competition, the **training set is comprised of the first 19 days of each month**, while the **test set is the 20th to the end of the month**. You must predict the **total count of bikes rented during each hour** covered by the test set, using only information available prior to the rental period.

Submissions are evaluated using the **Root Mean Squared Logarithmic Error (RMSLE)**.

Note: RMSLE is usually used when you don't want to penalize huge differences between the predicted and true values when both predicted and true values are huge numbers:
    - If both predicted and actual values are small: RMSE and RMSLE is same.
    - If either predicted or the actual value is big: RMSE > RMSLE
    - If both predicted and actual values are big: RMSE > RMSLE (RMSLE becomes almost negligible)

This is a **regression** task. 

Example regression algorithms/models:    
    - linear and polynomial regression 
    - decision trees and random forests 
    - gradient boosting 
    - neural networks


### Data description:
<pre>
datetime - hourly date + timestamp
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
<b>count - number of total rentals</b></pre>

# 1. Data exploration

## 1.1 Read data

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.metrics import mean_squared_log_error
import os

TRAIN_DATA = os.path.join('../input', 'train.csv')
TEST_DATA = os.path.join('../input', 'test.csv')

train_df = pd.read_csv(TRAIN_DATA)
test_df = pd.read_csv(TEST_DATA)
test_datetime = test_df['datetime']
print('Train data set size: {}'.format(train_df.shape))
print('Test data set size: {}'.format(test_df.shape))

print(train_df.dtypes)
print(test_df.dtypes)
print('N/A values in train data: \n{}'.format(train_df.isnull().sum()))

print('Randomly selected 5 rows from the dataframe:')
train_df.sample(frac=1).head(5)


## 1.2 Exploratory data analysis

In [ ]:
train_df.describe()

In [ ]:
i=0
for column in ['season', 'holiday', 'workingday', 'weather']:
    plt.subplot(1, 4, i+1)
    sn.countplot(column, data=train_df)
    i = i+1

## 1.3 Introduce new features

In [ ]:
# rental hour
train_df['hour'] = train_df['datetime'].apply(lambda x: x.split()[1].split(':')[0]).astype(int)
test_df['hour'] = test_df['datetime'].apply(lambda x: x.split()[1].split(':')[0]).astype(int)
# rental month
train_df['month'] = train_df['datetime'].apply(lambda x: x.split()[0].split('-')[1]).astype(int)
test_df['month'] = test_df['datetime'].apply(lambda x: x.split()[0].split('-')[1]).astype(int)
# day of the week
train_df['weekday'] = train_df['datetime'].apply(lambda x: datetime.datetime.strptime(x.split()[0].split(':')[0], '%Y-%m-%d').strftime('%w')).astype(int)
test_df['weekday'] = test_df['datetime'].apply(lambda x: datetime.datetime.strptime(x.split()[0].split(':')[0], '%Y-%m-%d').strftime('%w')).astype(int)


In [ ]:
sn.swarmplot(x='hour', y='temp', hue='season', data=train_df)

In [ ]:
sn.heatmap(train_df.corr())

We see high correlation between:
    - season and month
    - temperature and "feels like" temperature
    - number of casual/registered rentals and all rentals

It is also visible that the temperature is quite highly correlated with the number of rentals.

In [ ]:
CATEGORICAL_VARS = ['season', 'holiday', 'workingday', 'weather', 'hour', 'month', 'weekday']
for var in CATEGORICAL_VARS:
    train_df[var] = train_df[var].astype('category')

In [ ]:
# drop unnecessary columns
train_df = train_df.drop(['datetime', 'casual', 'registered'], axis=1)
test_df = test_df.drop(['datetime'], axis=1)

train_df.sample(frac=1).head(5)

y_train = train_df['count']
X_train = train_df.drop(['count'], axis=1)

# 2. Model training

## 2.1 Decision Tree regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz

decisionTree = DecisionTreeRegressor()

decisionTree.fit(X = X_train, y = y_train)
pred_tree = decisionTree.predict(X = X_train)

print("RMSLE on training data for a single Decision Tree with default parameters: {:.4f}".format(np.sqrt(mean_squared_log_error(y_train, pred_tree))))
# RMSLSE: 0.01078 - looks like a single Decision Tree with default parameters overfit to the train dataset
print('Probable overfitting')

decisionTree2 = DecisionTreeRegressor(max_depth=5)
decisionTree2.fit(X = X_train, y = y_train)
pred_tree2 = decisionTree2.predict(X = X_train)

print("RMSLE on training data for a single Decision Tree with maximum depth 5: {:.4f}".format(np.sqrt(mean_squared_log_error(y_train, pred_tree2))))

## 2.2 Random Forest regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
randomForest = RandomForestRegressor(n_estimators=100)

randomForest.fit(X = X_train, y = y_train)
pred_rf = randomForest.predict(X = X_train)

print("RMSLE on training data for Random Forest: {:.4f}".format(np.sqrt(mean_squared_log_error(y_train, pred_rf))))

## 2.3 Gradient Boosting regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=1000)
gbm.fit(X = X_train, y = np.log(y_train))
pred_gbm = gbm.predict(X = X_train)
print("RMSLE on training data for Gradient Boosting: ", np.sqrt(mean_squared_log_error(y_train, np.exp(pred_gbm))))

## 2.3 Predictions for test data

In [ ]:
predsTest = gbm.predict(X= test_df)
fig,(ax1,ax2)= plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sn.distplot(y_train, ax=ax1, bins=50)
sn.distplot(np.exp(predsTest), ax=ax2, bins=50)

In [ ]:
submission_df = pd.DataFrame({
        "datetime": test_datetime,
        "count": [max(0, x) for x in np.exp(predsTest)]
    })
submission_df.to_csv('../submission_gbm.csv', index=False)